In [2]:
!conda create -n supply-agent python=3.10 -y


Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/juilypachundkar/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/juilypachundkar/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.7.0



## Package Plan ##

  environment location: /Users/juilypachundkar/anaconda3/envs/supply-agent

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |      

In [3]:
!pip install pymysql python-dotenv pydantic orjson ollama


  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/0c/94/e4181a1f6286f545507528c78016e00065ea913276888db2262507693ce5/PyMySQL-1.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for ollama from https://files.pythonhosted.org/packages/be/f6/2091e50b8b6c3e6901f6eab283d5efd66fb71c86ddb1b4d68766c3eeba0f/ollama-0.5.3-py3-none-any.whl.metadata
  Obtaining dependency information for httpx>=0.27 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0


In [4]:
import os
os.environ["MYSQL_HOST"] = "supply-risk-db.c7a0kcm6iwtn.us-east-2.rds.amazonaws.com"
os.environ["MYSQL_USER"] = "admin"
os.environ["MYSQL_PASSWORD"] = "RDS2003!"
os.environ["MYSQL_DB"] = "supply_chain"
os.environ["USE_OLLAMA"] = "0"              # "1" to enable Ollama
os.environ["OLLAMA_MODEL"] = "qwen2.5:0.5b-instruct"


In [5]:
import pymysql, pandas as pd, os

conn = pymysql.connect(
    host=os.environ["MYSQL_HOST"],
    user=os.environ["MYSQL_USER"],
    password=os.environ["MYSQL_PASSWORD"],
    database=os.environ["MYSQL_DB"],
    cursorclass=pymysql.cursors.DictCursor,
)
with conn.cursor() as cur:
    cur.execute("SELECT * FROM supplier_risk_summary LIMIT 5;")
    rows = cur.fetchall()
conn.close()
pd.DataFrame(rows)


,supplier,orders,avg_lead_days,avg_ship_days,avg_defect_rate,avg_ship_cost,avg_total_cost,revenue,env_score,soc_score,gov_score,esg_score,risk_level
0,Supplier 3,15,14.33,5.20,2.466,4.79,468.80,97795.98,90,63,42,65,Medium
1,Supplier 1,27,16.78,6.07,1.804,5.51,574.85,157528.98,78,77,48,68,Medium
2,Supplier 5,18,14.72,6.22,2.665,5.79,536.02,110343.48,71,89,44,68,Low
3,Supplier 4,18,17.00,5.56,2.337,5.76,521.81,86468.97,49,72,45,55,High
4,Supplier 2,22,16.23,5.50,2.363,5.74,515.03,125467.45,84,94,43,74,Medium


In [7]:
import os, csv, datetime, json
from collections import defaultdict
from typing import Dict, Any

import pymysql
try:
    import ollama
except Exception:
    ollama = None

USE_OLLAMA = os.getenv("USE_OLLAMA", "0") == "1"
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "qwen2.5:0.5b-instruct")

MYSQL = dict(
    host=os.environ["MYSQL_HOST"],
    user=os.environ["MYSQL_USER"],
    password=os.environ["MYSQL_PASSWORD"],
    database=os.environ["MYSQL_DB"],
    cursorclass=pymysql.cursors.DictCursor,
)

def fetch_views():
    conn = pymysql.connect(**MYSQL)
    try:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM supplier_risk_summary;")
            risk = cur.fetchall()
            cur.execute("SELECT * FROM supplier_summary;")
            econ = cur.fetchall()
            # routes view is optional
            try:
                cur.execute("SELECT * FROM v_route_carrier_perf;")
                routes = cur.fetchall()
            except Exception:
                routes = []
    finally:
        conn.close()

    econ_by = {r["supplier"]: r for r in econ}
    routes_by = defaultdict(list)
    for r in routes:
        routes_by[r["supplier"]].append(r)
    return risk, econ_by, routes_by

def rule_agent(s: Dict[str, Any], revenue_share: float) -> Dict[str, Any]:
    triggers = []
    if s["risk_level"] == "High": triggers.append("High risk")
    if s["esg_score"] < 62: triggers.append("Low ESG")
    if float(s["avg_defect_rate"]) > 2.5: triggers.append("High defects")
    if float(s["avg_lead_days"]) > 16: triggers.append("Slow lead time")
    if revenue_share >= 0.25: triggers.append("Critical revenue dependency")

    recs = []
    if "High defects" in triggers:
        recs.append("Start 30-day CAPA; 100% incoming inspection for next 2 shipments (Owner: Quality).")
    if "Slow lead time" in triggers:
        recs.append("Negotiate expedited lanes; dual-source 20% with fastest alt (Owner: Procurement, 2 weeks).")
    if "Low ESG" in triggers:
        recs.append("Request ESG remediation plan & quarterly scorecards (Owner: Supplier Mgt, 3 weeks).")
    if "Critical revenue dependency" in triggers:
        recs.append("Business continuity plan; pre-qualify backup supplier (Owner: Category Mgr, 2 weeks).")
    if not recs:
        recs.append("Maintain supplier; monthly monitoring (Owner: Supplier Mgt).")

    return {
        "trigger_summary": ", ".join(triggers) or "Stable",
        "recommendation": " • ".join(recs),
        "confidence": 80 if triggers else 90
    }

SYSTEM = (
    "You are a supply chain risk copilot. Given a JSON payload of supplier metrics "
    "(risk_level, avg_defect_rate, avg_lead_days, esg_score, revenue_share, route/mode hints), "
    "return STRICT JSON with keys: trigger_summary, recommendation, confidence (0-100)."
)

def ollama_agent(payload: Dict[str, Any]) -> Dict[str, Any]:
    if ollama is None:
        raise RuntimeError("ollama not installed")
    r = ollama.chat(model=OLLAMA_MODEL, messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": json.dumps(payload, ensure_ascii=False)}
    ])
    text = r["message"]["content"].strip()
    # harden parsing
    try:
        return json.loads(text)
    except Exception:
        start, end = text.find("{"), text.rfind("}")
        if start >= 0 and end > start:
            return json.loads(text[start:end+1])
        return rule_agent(payload, payload.get("revenue_share", 0.0))

def run_agent(write_mysql=False) -> str:
    risk, econ_by, routes_by = fetch_views()
    total_rev = sum(float(econ_by.get(r["supplier"], {}).get("revenue", 0)) for r in risk) or 1.0
    out_rows = []

    for r in risk:
        sup = r["supplier"]
        rev = float(econ_by.get(sup, {}).get("revenue", 0))
        payload = {
            "supplier": sup,
            "risk_level": r["risk_level"],
            "avg_defect_rate": float(r["avg_defect_rate"]),
            "avg_lead_days": float(r["avg_lead_days"]),
            "esg_score": int(r["esg_score"]),
            "revenue": rev,
            "revenue_share": rev / total_rev,
            "hints": {"routes": routes_by.get(sup, [])[:2]},
        }
        if USE_OLLAMA:
            try:
                ai = ollama_agent(payload)
            except Exception:
                ai = rule_agent(payload, payload["revenue_share"])
        else:
            ai = rule_agent(payload, payload["revenue_share"])

        out_rows.append({
            "supplier": sup,
            "risk_level": r["risk_level"],
            "revenue": rev,
            "avg_defect_rate": r["avg_defect_rate"],
            "avg_lead_days": r["avg_lead_days"],
            "esg_score": r["esg_score"],
            "trigger_summary": ai["trigger_summary"],
            "recommendation": ai["recommendation"],
            "confidence": ai.get("confidence", 80),
        })

    fname = f"ai_recommendations_{datetime.date.today()}.csv"
    with open(fname, "w", newline="", encoding="utf-8") as f:
        import csv
        w = csv.DictWriter(f, fieldnames=list(out_rows[0].keys()))
        w.writeheader(); w.writerows(out_rows)
    print("Wrote:", fname)

    if write_mysql:
        conn = pymysql.connect(**MYSQL)
        with conn.cursor() as cur:
            cur.execute("CREATE TABLE IF NOT EXISTS ai_recommendations ("
                        "id BIGINT AUTO_INCREMENT PRIMARY KEY,"
                        "supplier VARCHAR(100), risk_level VARCHAR(10), revenue DECIMAL(14,2),"
                        "avg_defect_rate DECIMAL(6,3), avg_lead_days DECIMAL(6,2), esg_score INT,"
                        "trigger_summary VARCHAR(255), recommendation TEXT, confidence INT,"
                        "created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)")
            cur.execute("TRUNCATE ai_recommendations;")
            cur.executemany("""
                INSERT INTO ai_recommendations
                (supplier,risk_level,revenue,avg_defect_rate,avg_lead_days,esg_score,trigger_summary,recommendation,confidence)
                VALUES (%(supplier)s,%(risk_level)s,%(revenue)s,%(avg_defect_rate)s,%(avg_lead_days)s,%(esg_score)s,%(trigger_summary)s,%(recommendation)s,%(confidence)s)
            """, out_rows)
        conn.commit(); conn.close()
        print("Loaded ai_recommendations table.")
    return fname

# run it
csv_path = run_agent(write_mysql=False)
csv_path


Wrote: ai_recommendations_2025-08-10.csv


'ai_recommendations_2025-08-10.csv'